# Clarity Model Evaluation (2-GPU)
Loads the model distributed across 2 GPUs using FP16 and runs inference on the HuggingFace test dataset.

In [ ]:
# Install dependencies if needed
%pip install transformers accelerate datasets scikit-learn -q

In [ ]:
import pandas as pd
import torch
import re
import warnings
from tqdm.notebook import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from sklearn.metrics import (
    accuracy_score, 
    precision_recall_fscore_support, 
    f1_score, 
    classification_report, 
    confusion_matrix
)

# Suppress attention mask warning
warnings.filterwarnings("ignore", message=".*attention_mask.*")

# Check available GPUs
print(f"Available GPUs: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    props = torch.cuda.get_device_properties(i)
    print(f"  GPU {i}: {props.name} ({props.total_memory / 1024**3:.1f} GB)")

In [ ]:
# ==================== CONFIGURATION ====================
# Change these as needed

MODEL_NAME = "Frenzyknight/Clarity-llama-70b"  # Your model on HuggingFace
DATASET_NAME = "Frenzyknight/clarity-dataset"  # Your HuggingFace dataset
QEVASION_DATASET = "ailsntua/QEvasion"  # For ground truth labels

OUTPUT_PATH = "evaluation_2gpu_results.csv"  # Output file
MAX_NEW_TOKENS = 600  # Max tokens to generate
TEMPERATURE = 0.1  # Generation temperature
MAX_INPUT_LENGTH = 3500  # Max input tokens
LIMIT = None  # Set to a number for testing (e.g., 10), None for full run

In [ ]:
# ==================== HELPER FUNCTIONS ====================
def extract_label(response: str) -> str:
    """Extract label from COT response."""
    # Look for explicit LABEL: pattern
    match = re.search(r'LABEL:\s*(Clear Reply|Clear Non-Reply|Ambivalent)', response, re.IGNORECASE)
    if match:
        label = match.group(1).strip().lower()
        if "non-reply" in label:
            return "Clear Non-Reply"
        elif "clear reply" in label:
            return "Clear Reply"
        elif "ambivalent" in label:
            return "Ambivalent"
    
    # Fallback: check last 100 chars
    last_part = response[-100:].lower() if len(response) > 100 else response.lower()
    if "clear non-reply" in last_part or "non-reply" in last_part:
        return "Clear Non-Reply"
    elif "clear reply" in last_part:
        return "Clear Reply"
    elif "ambivalent" in last_part:
        return "Ambivalent"
    
    return "PARSE_ERROR"


print("✅ Helper functions defined")

In [ ]:
# ==================== LOAD MODEL (2 GPUs) ====================
print("=" * 60)
print(f"Loading model: {MODEL_NAME}")
print("Distributing across available GPUs with FP16...")
print("=" * 60)

# Load tokenizer
print("\nLoading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model distributed across GPUs
print("Loading model with device_map='auto' (distributes across all GPUs)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",  # Automatically distribute layers across GPUs
    torch_dtype=torch.float16,  # FP16 for efficiency
    trust_remote_code=True,
)

print("\n✅ Model loaded successfully!")
if hasattr(model, 'hf_device_map'):
    # Show how model is distributed
    devices = set(model.hf_device_map.values())
    print(f"Model distributed across devices: {devices}")
    # Count layers per device
    device_counts = {}
    for layer, device in model.hf_device_map.items():
        device_counts[device] = device_counts.get(device, 0) + 1
    for device, count in sorted(device_counts.items()):
        print(f"  Device {device}: {count} modules")

In [ ]:
# ==================== LOAD TEST DATASET ====================
print(f"Loading test dataset from: {DATASET_NAME}")
test_dataset = load_dataset(DATASET_NAME, split="test")
print(f"✅ Loaded {len(test_dataset)} test examples")
print(f"   Columns: {test_dataset.column_names}")

# Load ground truth labels from QEvasion
print(f"\nLoading ground truth from: {QEVASION_DATASET}")
qevasion_test = load_dataset(QEVASION_DATASET, split="test")
print(f"✅ Loaded {len(qevasion_test)} examples with labels")

# Apply limit if set
if LIMIT:
    print(f"\n⚠️  Limiting to first {LIMIT} examples (for testing)")
    test_dataset = test_dataset.select(range(min(LIMIT, len(test_dataset))))
    qevasion_test = qevasion_test.select(range(min(LIMIT, len(qevasion_test))))

In [ ]:
# ==================== RUN INFERENCE ====================
results = []
total = len(test_dataset)

print("\n" + "=" * 60)
print(f"Starting inference on {total} examples")
print("=" * 60 + "\n")

for idx, example in tqdm(enumerate(test_dataset), total=total, desc="Evaluating"):
    # Get conversations from dataset (already formatted as system + user messages)
    messages = example["conversations"]
    
    # Format messages to string, then tokenize
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    
    # Tokenize the prompt string
    encoded = tokenizer(prompt, return_tensors="pt")
    inputs = encoded["input_ids"]
    input_len = inputs.shape[-1]
    
    # Skip if too long
    if input_len > MAX_INPUT_LENGTH:
        tqdm.write(f"[{idx + 1}/{total}] SKIPPED (input too long: {input_len} tokens)")
        results.append({
            "idx": idx,
            "prediction": "SKIPPED",
            "raw_output": f"Input too long: {input_len} tokens"
        })
        continue
    
    try:
        # Move to model's device
        inputs = inputs.to(model.device)
        attention_mask = torch.ones_like(inputs)
        
        # Generate
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs,
                attention_mask=attention_mask,
                max_new_tokens=MAX_NEW_TOKENS,
                temperature=TEMPERATURE,
                do_sample=TEMPERATURE > 0,
                use_cache=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        
        # Decode only the new tokens
        full_response = tokenizer.decode(
            outputs[0, input_len:],
            skip_special_tokens=True
        ).strip()
        
        # Extract label
        prediction = extract_label(full_response)
        
        results.append({
            "idx": idx,
            "prediction": prediction,
            "raw_output": full_response
        })
        
    except Exception as e:
        tqdm.write(f"[{idx + 1}/{total}] ERROR: {e}")
        results.append({
            "idx": idx,
            "prediction": "ERROR",
            "raw_output": str(e)
        })
    
    # Clear CUDA cache periodically
    if (idx + 1) % 25 == 0:
        torch.cuda.empty_cache()

print("\n✅ Inference complete!")

In [ ]:
# ==================== CREATE RESULTS DATAFRAME ====================
results_df = pd.DataFrame(results)

# Add ground truth labels
results_df["clarity_label"] = [qevasion_test[i]["clarity_label"] for i in range(len(results_df))]
results_df["annotator1"] = [qevasion_test[i]["annotator1"] for i in range(len(results_df))]
results_df["annotator2"] = [qevasion_test[i]["annotator2"] for i in range(len(results_df))]
results_df["annotator3"] = [qevasion_test[i]["annotator3"] for i in range(len(results_df))]

# Save results
results_df.to_csv(OUTPUT_PATH, index=False)
print(f"✅ Saved results to {OUTPUT_PATH}")
print(f"\nDataFrame shape: {results_df.shape}")
results_df.head()

In [ ]:
# ==================== FILTER VALID PREDICTIONS ====================
y_true = results_df["clarity_label"].str.strip()
y_pred = results_df["prediction"].str.strip()

VALID_LABELS = {"Clear Reply", "Clear Non-Reply", "Ambivalent"}
mask = y_true.isin(VALID_LABELS) & y_pred.isin(VALID_LABELS)

y_true_filtered = y_true[mask]
y_pred_filtered = y_pred[mask]

print(f"Valid predictions: {mask.sum()} / {len(mask)}")
print(f"Filtered out: {(~mask).sum()} (PARSE_ERROR, SKIPPED, ERROR)")

# Show breakdown of filtered
invalid_preds = y_pred[~mask].value_counts()
if len(invalid_preds) > 0:
    print("\nInvalid prediction breakdown:")
    for label, count in invalid_preds.items():
        print(f"  {label}: {count}")

In [ ]:
# ==================== COMPUTE METRICS ====================
accuracy = accuracy_score(y_true_filtered, y_pred_filtered)

precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
    y_true_filtered,
    y_pred_filtered,
    average="macro"
)

f1_weighted = f1_score(
    y_true_filtered,
    y_pred_filtered,
    average="weighted"
)

print("=" * 60)
print("EVALUATION METRICS")
print("=" * 60)
print(f"\nAccuracy        : {accuracy:.4f}")
print(f"Precision (Mac) : {precision_macro:.4f}")
print(f"Recall (Mac)    : {recall_macro:.4f}")
print(f"F1 (Macro)      : {f1_macro:.4f}")
print(f"F1 (Weighted)   : {f1_weighted:.4f}")

In [ ]:
# ==================== CLASSIFICATION REPORT ====================
print("\n" + "=" * 60)
print("DETAILED CLASSIFICATION REPORT")
print("=" * 60 + "\n")

print(classification_report(
    y_true_filtered,
    y_pred_filtered,
    digits=4
))

In [ ]:
# ==================== CONFUSION MATRIX ====================
labels = ["Clear Reply", "Clear Non-Reply", "Ambivalent"]
cm = confusion_matrix(y_true_filtered, y_pred_filtered, labels=labels)

print("\n📈 Confusion Matrix:")
print(f"{'':20} {'Pred CR':>12} {'Pred CNR':>12} {'Pred Amb':>12}")
print("-" * 60)
for i, true_label in enumerate(labels):
    row = f"{true_label:20}"
    for j in range(3):
        row += f" {cm[i][j]:>12}"
    print(row)

In [ ]:
# ==================== DISTRIBUTION COMPARISON ====================
print("\n📊 Prediction vs Ground Truth Distribution:")
print("\nGround Truth:")
for label in labels:
    count = (y_true_filtered == label).sum()
    pct = count / len(y_true_filtered) * 100
    print(f"   {label}: {count} ({pct:.1f}%)")

print("\nPredictions:")
for label in labels:
    count = (y_pred_filtered == label).sum()
    pct = count / len(y_pred_filtered) * 100
    print(f"   {label}: {count} ({pct:.1f}%)")

In [ ]:
# ==================== SAMPLE PREDICTIONS ====================
print("\n🔍 Sample Predictions (first 10):")
print("-" * 80)
for i in range(min(10, len(results_df))):
    row = results_df.iloc[i]
    pred = row['prediction']
    true = row['clarity_label']
    match = "✓" if pred == true else "✗"
    raw_preview = str(row['raw_output'])[:60] + "..." if len(str(row['raw_output'])) > 60 else row['raw_output']
    print(f"{i+1}. {match} Pred: {pred:15} | True: {true:15}")
    print(f"   Raw: {raw_preview}")
    print()

In [ ]:
# ==================== CLEANUP ====================
# Free GPU memory
del model
torch.cuda.empty_cache()
print("✅ GPU memory cleared")